In [18]:
import pandas as pd
data = pd.read_excel("chatbot_data11111111111111.xlsx")
# print(data)

In [3]:
# Pattern ARRAYS
PII_PATTERNS = [
    r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}",             # Email
    r"(?:\+?\d{1,3}[\s\-]?)?(?:\(?\d{2,4}\)?[\s\-]?)?\d{6,10}",    # Phone (global)
    r"\b[A-Z]{5}[0-9]{4}[A-Z]\b",                                  # PAN (India)
    r"\b\d{4}[\s\-]?\d{4}[\s\-]?\d{4}\b",                          # Aadhaar (India)
    r"\b(?:\d{4}[\s\-]?){4}\b",                                    # ATM/Credit/Debit card
]

SECRET_PATTERNS = [
    r"\b[A-Za-z0-9_\-]{20,}\b",                                    # API / Token / Secret
]

In [6]:
# Combine arrays → Single Regex
PII_REGEX = "(" + "|".join(PII_PATTERNS) + ")"
SECRET_REGEX = "(" + "|".join(SECRET_PATTERNS) + ")"
ALL_SECURE_REGEX = "(" + "|".join(PII_PATTERNS + SECRET_PATTERNS) + ")"

In [5]:
datas = pd.read_excel("chatbot_data11111111111111.xlsx")

In [19]:
# Auto Detect + Extract Values (NO LOOPS)
datas = datas.assign(
    has_secure_info = datas["user"].str.contains(ALL_SECURE_REGEX, regex=True, na=False),
    pii_values    = datas["user"].str.findall(PII_REGEX),
    secret_values = datas["user"].str.findall(SECRET_REGEX),
    has_email  = datas["user"].str.contains(PII_PATTERNS[0], regex=True, na=False),
    has_phone  = datas["user"].str.contains(PII_PATTERNS[1], regex=True, na=False),
    has_pan    = datas["user"].str.contains(PII_PATTERNS[2], regex=True, na=False),
    has_aadhar = datas["user"].str.contains(PII_PATTERNS[3], regex=True, na=False),
    has_card   = datas["user"].str.contains(PII_PATTERNS[4], regex=True, na=False),
    has_secret = datas["user"].str.contains(SECRET_PATTERNS[0], regex=True, na=False),
)

In [9]:
# Data Masking
datas["masked_user"] = (
    datas["user"]
    .str.replace(PII_REGEX, "[PII_MASKED]", regex=True)
    .str.replace(SECRET_REGEX, "[SECRET_MASKED]", regex=True)
)

In [11]:
# Risk Score
datas["risk_score"] = ( datas["has_email"].astype(int)  * 2+ datas["has_phone"].astype(int)  * 2 + datas["has_pan"].astype(int)    * 5
    + datas["has_aadhar"].astype(int) * 5+ datas["has_card"].astype(int)   * 5  + datas["has_secret"].astype(int) * 4 )

In [12]:
# Risk Level
datas["risk_level"] = pd.cut(
    datas["risk_score"],
    bins=[-1, 0, 3, 7, 100],
    labels=["NaN", "Low", "Medium", "High"]
)

In [15]:
# Sort by Risk Score (DESC)
datas_sorted = datas.sort_values(by="risk_score", ascending=False)

In [16]:
# Save Outputs - Excel file
datas.to_excel("chatbot_secure_output.xlsx", index=False)
datas_sorted.to_excel("chatbot_secure_sorted_by_risk.xlsx", index=False)

In [20]:
# Preview
print(datas_sorted[["user", "masked_user", "risk_score", "risk_level", "pii_values", "secret_values"]].head(10))